<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li><li><span><a href="#Общий-вывод-по-проекту" data-toc-modified-id="Общий-вывод-по-проекту-6"><span class="toc-item-num">6&nbsp;&nbsp;</span><b>Общий вывод по проекту</b></a></span></li><li><span><a href="#Общий-вывод-по-проекту-V2" data-toc-modified-id="Общий-вывод-по-проекту-V2-7"><span class="toc-item-num">7&nbsp;&nbsp;</span><b>Общий вывод по проекту V2</b></a></span></li><li><span><a href="#Общий-вывод-по-проекту-V3" data-toc-modified-id="Общий-вывод-по-проекту-V3-8"><span class="toc-item-num">8&nbsp;&nbsp;</span><b>Общий вывод по проекту V3</b></a></span></li><li><span><a href="#Общий-вывод-по-проекту-V4" data-toc-modified-id="Общий-вывод-по-проекту-V4-9"><span class="toc-item-num">9&nbsp;&nbsp;</span><b>Общий вывод по проекту V4</b></a></span></li><li><span><a href="#Общий-вывод-по-проекту-V5" data-toc-modified-id="Общий-вывод-по-проекту-V5-10"><span class="toc-item-num">10&nbsp;&nbsp;</span><b>Общий вывод по проекту V5</b></a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

STATE=12345 # запишу константу random_state

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
def dano(df):
    display(df.head())
    display(df.info(),
            'Количество пропусков:', df.isna().sum(),         
           )

In [4]:
dano(data)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

'Количество пропусков:'

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

**Вывод**
- 5000 строк, без пропусков
- Названия столбцов записанны кирилицей и с верхним регистром
- Типы данных соответствуют данным в столбцах

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

In [5]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']
r2 = []

In [6]:
#создам функцию дл вывода метрики R2
def r2_model(features,target):
    features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                            target,
                                                                            test_size=0.3, 
                                                                            random_state=STATE)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_test)
    r2.append(r2_score(target_test, predictions))
    return r2_score(target_test, predictions)

In [7]:
print("Показатель метрики R2 модели с обычными данными:")
r2_model(features, target)

Показатель метрики R2 модели с обычными данными:


0.4305278542485165

Создам случайную матрицу, такой же размерности как и матрица признаков

In [8]:
matrix = np.random.rand(4,4)
# Проверим обратимая ли матрица получиллась
matrix @ np.linalg.inv(matrix)

array([[ 1.00000000e+00, -5.06487359e-17, -3.10676638e-17,
         1.28406032e-16],
       [ 2.28596121e-16,  1.00000000e+00,  3.10110950e-17,
         1.32217974e-16],
       [ 1.46963815e-17,  2.49961611e-17,  1.00000000e+00,
        -1.89468615e-17],
       [-2.55243893e-17,  1.23088959e-17, -9.05035891e-19,
         1.00000000e+00]])

При умножении матрицы на обратную, на главной диагонали единицы - значит марица обратимая

In [9]:
o_matrix = features.values @ matrix

In [10]:
print("Показатель метрики R2 на данных умноженных на обратимую матрицу")
r2_model(o_matrix, target)

Показатель метрики R2 на данных умноженных на обратимую матрицу


0.43052785424844753

**Ответ:** Качество линейной регресии не поменялось

**Обоснование:** При умножения матрицы признаков на случайную обратимую матрицу, метрика R2 оказалась без изменений

Предсказания:

$
a = Xw
$

Задача обучения:

$
w = \arg\min_w MSE(Xw, y)
$

Формула обучения:

$
w = (X^T X)^{-1} X^T y
$

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Для проверки работы формулы предсказания создадим рандомные матрицы и векторы и пошагово просмотрим все шаги работы линейной регресии

In [11]:
# Создание матриц
X = np.random.rand(4,2)
P = np.random.rand(2,2)
X1 = X.dot(P)

In [12]:
# Создание вектора
y = np.random.rand(2,1)
y = np.array(y)
y

array([[0.14584327],
       [0.0464553 ]])

Подставим в формулу предсказания $a$ формулу вектора весов $w$
$$a = X (X^T X)^{-1} X^T y$$
Выполним все действия пошагово

In [13]:
X.T @ X

array([[2.1331655 , 1.30482281],
       [1.30482281, 1.18507871]])

In [14]:
np.linalg.inv(X.T @ X)

array([[ 1.4357517 , -1.58082459],
       [-1.58082459,  2.58438191]])

In [15]:
np.linalg.inv(X.T @ X) @ X.T @ X

array([[ 1.00000000e+00, -1.23090137e-17],
       [-8.52464349e-16,  1.00000000e+00]])

Как видим все матрицы сокращаются до единичной и последнее действие получается умножение единичной матрицы на вектор целевого признака

In [16]:
a = (np.linalg.inv(X.T @ X) @ X.T @ X) @ y
a

array([[0.14584327],
       [0.0464553 ]])

In [17]:
a1 = (np.linalg.inv(X1.T @ X1) @ X1.T @ X1) @ y
a1

array([[0.14584327],
       [0.0464553 ]])

Даже при изменении данной матрицы - предсказание зависит только от целевого признака

Предсказания:

$
a = Xw
$

Задача обучения:

$
w = \arg\min_w MSE(Xw, y)
$

Формула обучения:

$
w = (X^T X)^{-1} X^T y
$

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Для проверки работы формулы предсказания создадим рандомные матрицы и векторы и пошагово просмотрим все шаги работы линейной регресии

И так для проверки алгоритма вычисления мы должны подставить матрицу $P$ в наши формулы и сократить все возможные элементы:
$$w′=((XP)^{T}XP)^{−1}(XP)^{T}y$$ из чего следует: $$w′=P^{−1}(X^{T}X)^{−1}(P^{T})^{−1}P^{T}X^{T}y$$
Так как матрица $P$ у нас случаная и обратимая получается в формуле выделить два имеющихся равенства:
$$(P^{T})^{−1}P^{T} = E$$ и после такой замены выйде: $$w′=P^{−1}(X^{T}X)^{−1}E X^{T}y$$
При умножении матрицы на единичную матрицу произведение равно самой матрице, значит второе равенство получается таким:
$$(X^{T}X)^{−1} X^{T}y = w$$
Значит $$w′ = P^{−1}w$$
А предсказание $$ a′ = X P w′ = X P P^{−1}w$$ где снова получается единичная матрица и в итоге 
$$a′=a$$


In [18]:
# Создание матриц
#X = np.random.rand(4,2)
#P = np.random.rand(2,2)
#X1 = X.dot(P)

In [19]:
# Создание вектора
#y = np.random.rand(2,1)
#y = np.array(y)
#y

array([[0.18300836],
       [0.34907624]])

Подставим в формулу предсказания $a$ формулу вектора весов $w$
$$a = X (X^T X)^{-1} X^T y$$
Выполним все действия пошагово

In [20]:
## 1 Действие в скобках
#X.T @ X

array([[1.31086325, 0.79854394],
       [0.79854394, 0.68463233]])

<div class="alert alert-block alert-danger">
<b>Ошибка❌ V2:</b> 
    
Хмм, а ты точно уверен, что первым шагом будет умножение матриц?)
</div>

<div class="alert alert-block alert-info">
<b>Комментарий студента V2:</b>
     
Я делаю умножение матриц в скобках, что бы потом матрицу, полученную из их произведения сделать обратной и далее умножать поочереди на матрицу и транспортированную матрицу( не имеет значения порядок умножения по законам математики)
    
p.s. добавил коментарий к действиям     
</div>


<div class="alert alert-block alert-danger">
<b>Ошибка❌ V3:</b> 
    
Окей, хорошо, но давай посмотрим на то, как формула выполняется:
    
- $$X * (X^T * X)^{-1} * X^T * y = X * X^{-1} * (X^T)^{-1} * X^T * y $$
- По определению мы знаем, что у неквадратных матриц обратных матриц не существует
- Получается, что само действие X^{-1} неверно, и в коде должно вызвать ошибку
- попробуй выполнить np.linalg.inv(X.T) либо np.linalg.inv(X) (у тебя выйдет ошибка), так как np.linalg.inv(X.T @ X) по своему алгоритму первым делом помножит матрицы друг с другом, что по нашим действиям выше неверно (первым делом идет поиск обратимой матрицы)

    
    
- В итоге наша формула весов (где признаки помножены на случайную обратную матрицу) из такой $$w′=((XP)^{T}XP)^{−1}(XP)^{T}y$$    
- Превращается в такую $$w′=P^{−1}(X^{T}X)^{−1}(P^{T})^{−1}P^{T}X^{T}y$$. И в заключении имеем $$w′=P^{−1}w$$, которое потом подставляем в формулу предсказания $$a=Xw = X*P*P^{−1}w = X*E*w$$, что и требовалось доказать
</div>

<div class="alert alert-block alert-info">
<b>Комментарий студента V3:</b>
     
Но почему формула выполняется именно так, а не по другому?ведь "от перестановки мест множителей произведение не изменяется", тогда формулу можно выполнять по другому 
    $$X * (X^T * X)^{-1} * X^T * y = X * X^T * (X^T * X)^{-1} * y $$
А так как обратная матрица может быть только у квадратичной матрицы, нельзя вытаскивать из под скобки каждую матрицу по отдельности, потому что именно произведение матрицы на свои транспортировку дасть квадратичную матрицу, что требует формула
</div>


<div class="alert alert-block alert-danger">
<b>Ошибка❌ V4:</b> 
    
Ну смотри:
    
1)  "от перестановки мест множителей произведение не изменяется" - данное правило работает только для чисел, а не для матриц, ведь матрицы некоммутативны по основным свойствам матричного произведения ( вот тебе материал для ознакомления, будет интересно) - https://www.berdov.com/works/matrix/umnozhenie-matric/)
    
2) "что требует формула" - формула требует прийти к тому, чтобы предсказание не изменилось: $$a=Xw$$, то есть при умножении матрицы признаков на случайную обратимую матрицу мы не получим другой результат
</div>

<div class="alert alert-block alert-info">
<b>Комментарий студента V4:</b>
     
Я вчера после отправки проекта тебе потом задумался и пробовал разные варианты решения этого уравнения, как оказалось моем варианте - это какой то частный случай, где должны совпасть определенные условия, такие как :размерность матрицы должна быть строго количество строк больше, чем столбцов, и "у" - должен быть специфичный - размерностью равной количеству столбцов, так что я теперь буду предерживаться ТВОЕМУ решению, но мне не понятен момент  $$w′=P^{−1}(X^{T}X)^{−1}(P^{T})^{−1}P^{T}X^{T}y$$. куда деваются тут матрицы X, не вижу варианта их сокращения, разве не останется один из Х? $$w′=P^{−1}w$$
    
</div>


<div class="alert alert-block alert-success">
<b>Успех👍 V5:</b> 
    
Миша, ты красавчик, честно, потому что держаться за свою позицию важно и ты правильно все делал). Я рад, что смог хотя бы направить тебя в сторону более верного варианта решения.
    
По поводу твоего вопроса - в данный момент мы можем провести перевод в единичную матрицу данных слогаемых: $$(P^{T})^{−1}P^{T}$$, в итоге у нас остается следующее: $$ w′=P^{−1}(X^{T}X)^{−1}EX^{T}y = P^{−1}(X^{T}X)^{−1}X^{T}y $$. и здесь можно заметить, что у нас в формуле появляется еще одна формула: $$ w=(X^{T}X)^{−1}X^{T}y $$. Ну а далее мы заменяем: $$(X^{T}X)^{−1}X^{T}y$$ на: $$ w$$ и получаем: $$w′=P^{−1}w$$. А в конце ты уже все прекрасно знаешь)
</div>

In [21]:
# 3 и 4 действия - умножение на матрицу и транспортированную матрицу
#от перестановки местами множителей произведение не меняется
#np.linalg.inv(X.T @ X) @ X.T @ X

array([[ 1.00000000e+00, -1.14586268e-16],
       [ 8.68396677e-17,  1.00000000e+00]])

Как видим все матрицы сокращаются до единичной и последнее действие получается умножение единичной матрицы на вектор целевого признака

In [22]:
#a= (np.linalg.inv(X.T @ X) @ X.T @ X) @ y
#a

array([[0.18300836],
       [0.34907624]])

In [23]:
#a1 = (np.linalg.inv(X1.T @ X1) @ X1.T @ X1) @ y
#a1

array([[0.18300836],
       [0.34907624]])

Даже при изменении данной матрицы - предсказание зависит только от целевого признака

## Алгоритм преобразования

**Алгоритм**
$$
((features.values)*100) @ matrix
$$

**Обоснование**

Как бы мы не меняли наши принзаки качество линейной регрессии будет одинаковое, так как вектор весов линейной регрессии в новой матрице будет соответствовать весу в исходной.
<br> Проверим наши слова:

## Проверка алгоритма

In [24]:
new_matrix = ((𝑓𝑒𝑎𝑡𝑢𝑟𝑒𝑠.𝑣𝑎𝑙𝑢𝑒𝑠)*100)@𝑚𝑎𝑡𝑟𝑖𝑥

In [25]:
print("Показатель метрики R2 на данных с придуманным алгоритмом")
r2_model(new_matrix, target)

Показатель метрики R2 на данных с придуманным алгоритмом


0.4305278542484081

In [26]:
r2_taple=pd.DataFrame(r2, index=['features', 'features.values @ matrix', 'algoritm'])

In [27]:
r2_taple

,0
features,0.430528
features.values @ matrix,0.430528
algoritm,0.430528


**Вывод** 
<br> R2 метрика при всех изменениях признаков данных ни как не поменялась, это связанно с тем что при всех изенениях признаков Предсказание в линейной регрессии вычисляется по формуле 
$$
a = Xw
$$
- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу), который считается по формуле 
$$
w = (X^T X)^{-1} X^T y
$$

- где $y$ — вектор целевого признака
- а $P$ — матрица, на которую умножаются признаки

Итоговая формула для измененого предсказания будет выглядеть так:
$$
a′ = XPw′=((𝑋𝑃)^𝑇𝑋𝑃)^{−1}(𝑋𝑃)^𝑇𝑦
$$
после всех сокращений даст ответ 
$$
a′ = Xw = a
$$